In [ ]:
"""This library queries the YouTube API for the list of videos on any given channel and returns the video ids, details,
and stats as a pandas dataframe that can be exported to a .csv

Needs modifications for efficiency and replication
"""

In [22]:
#Import necessary libraries and load api keys
import os
import time
import sys
import json
import requests
import datetime
from collections import OrderedDict
import pandas as pd

key_file_loc = "C:/Users/CodeAlias/Desktop/tweepy_auth.txt"

with open(key_file_loc) as f: keys = dict(x.rstrip().split(None,1) for x in f)
    
key = keys['ytapi']

ids = [] # or load channel ids

In [45]:
def run_comments(http_endpoint, 
                 next_page_token=False, stop_after_n_iteration=1**10,
                 cutoff_date=datetime.datetime(1990,1,1),
                 verbose=1):
    
    comment_ids = []
    reply_list = []
    iterations = 0
    run = True
    while run:
        if iterations > stop_after_n_iteration: run = False
        
        
        if next_page_token:
            http_endpoint += "&pageToken={}".format(next_page_token)

        response = requests.get(http_endpoint)
        response_json = load_response(response, verbose)
        if response_json:
            for item in response_json['items']:
                c_id = parse_comment_metadata(item)
                reply_list.append(c_id['comment_id'])
                if c_id['publish_date'] <= cutoff_date: 
                    run = False
                    break 
                if not return_date:
                    c_id = c_id['comment_id']
                comment_ids.append(c_id)                

            try: 
                next_page_token = response_json['nextPageToken']
                iterations += 1
                log(">> {} comments parsed. Next Token = {}".format(len(commentIds), next_page_token),
                   verbose=verbose)
            except:
                run = False
        time.sleep(.1)
        
        return reply_list, comment_ids
    

In [46]:
def get_video_comments(video_id, key, 
                       next_page_token=False,
                       stop_after_n_iteration=1**10,
                       cutoff_date=datetime.datetime(1990,1,1),
                       return_date=True,
                       verbose=1):
    
    if not isinstance(video_id, str):
        raise Exception("Only string values permitted")
        
    init_http_endpoint = ("https://www.googleapis.com/youtube/v3/commentThreads?"
                         "part=snippet&textFormat=plainText&maxResults=100&"
                         "videoId={}&key={}".format(video_id,key))
    
    reply_list, comment_ids = run_comments(init_http_endpoint, 
                                           next_page_token, 
                                           stop_after_n_iteration, 
                                           cutoff_date, 
                                           return_date, verbose)
    
    
    for reply_id in reply_list:
        http_endpoint = ("https://www.googleapis.com/youtube/v3/comments?"
                         "part=snippet&textFormat=plainText&maxResults=100&"
                         "parentId={}&key={}".format(reply_id,key))
        
        trash, replies = run_comments(http_endpoint, 
                                     next_page_token,
                                     stop_after_n_iteration,
                                     cutoff_date,
                                     return_date, verbose)
        
        comment_ids.extend(replies)
        
        
    return comment_ids

In [49]:
def parse_comment_metadata(item):
    print(item)
    
    if item['snippet'].get('topLevelComment'):
        item = item['snippet']['topLevelComment']
        
    comment_meta = OrderedDict(
        
        channel_id = item["snippet"].get("authorChannelUrl"),
        channel_image_url = item["snippet"].get("authorProfileImageUrl"),
        channel_display_name = item['snippet'].get('authorDisplayName'),
        comment_id = item.get("id"),
        comment_like_count = item["snippet"].get("likeCount"),
        publish_date = parse_yt_datetime(item["snippet"].get("publishedAt")),
        text_display = item["snippet"].get("textDisplay"),
        video_id = item["snippet"].get("videoId"),
        viewer_rating = item["snippet"].get("viewerRating"),
        parent_id = item["snippet"].get("parentId"),
        collection_date = datetime.datetime.now() #TODO
        
    )
    
    return comment_meta

